In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import string
pd.options.mode.chained_assignment = None

In [2]:
import os
os.getcwd()

'C:\\Users\\JiatingChen\\Documents\\nlp-coe\\twitter-conversational-chatbot\\EDA Notebooks'

In [3]:
os.listdir('C:\\Users\\JiatingChen\\Documents\\archive\\twcs')

['twcs.csv']

In [4]:
full_df = pd.read_csv("C:\\Users\\JiatingChen\\Documents\\archive\\twcs\\twcs.csv")

In [5]:
full_df["text"] = full_df["text"].astype(str)

In [6]:
full_df.shape #2,811,774

(2811774, 7)

In [7]:
full_df.sort_values('tweet_id').head(20)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
2314173,7,115712,True,Tue Oct 31 21:47:48 +0000 2017,@sprintcare the only way I can get a response ...,NaN,6.0
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN
2314166,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
2314174,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0


In [8]:
# in_response_tweet_id = 1: tweet_id = 1 is the prior sentense
# response_tweet_id = 1: tweet_id = 1 is the next sentense

In [9]:
tweets = full_df

In [10]:
first_inbound = tweets[pd.isnull(tweets.in_response_to_tweet_id)] #initize the conversation
inbounds_and_outbounds = pd.merge(first_inbound, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id') 
inbounds_and_outbounds.sort_values(['tweet_id_x', 'created_at_y']).shape #895,514 back-and-forth dialog

(895514, 14)

In [11]:
inbounds_and_outbounds.head()

,tweet_id_x,author_id_x,inbound_x,created_at_x,text_x,response_tweet_id_x,in_response_to_tweet_id_x,tweet_id_y,author_id_y,inbound_y,created_at_y,text_y,response_tweet_id_y,in_response_to_tweet_id_y
0,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 Can you please send us a private messa...,"5,7",8.0
1,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,9,sprintcare,False,Tue Oct 31 21:46:14 +0000 2017,@115712 I would love the chance to review the ...,NaN,8.0
2,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,10,sprintcare,False,Tue Oct 31 21:45:59 +0000 2017,@115712 Hello! We never like our customers to ...,NaN,8.0
3,18,115713,True,Tue Oct 31 19:56:01 +0000 2017,@115714 y’all lie about your “great” connectio...,17,NaN,17,sprintcare,False,Tue Oct 31 19:59:13 +0000 2017,@115713 H there! We'd definitely like to work ...,16,18.0
4,20,115715,True,Tue Oct 31 22:03:34 +0000 2017,"@115714 whenever I contact customer support, t...",19,NaN,19,sprintcare,False,Tue Oct 31 22:10:10 +0000 2017,@115715 Please send me a private message so th...,NaN,20.0


In [12]:
inbounds_and_outbounds[['text_x', 'text_y']].head(100).to_csv('conv.txt')

In [13]:
first_inbound2 = tweets[pd.isnull(tweets.in_response_to_tweet_id) & tweets.inbound]
inbounds_and_outbounds = pd.merge(first_inbound2, tweets, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id') #.sample(frac=1)
inbounds_and_outbounds2 = inbounds_and_outbounds[inbounds_and_outbounds.inbound_y ^ True]
inbounds_and_outbounds2.sort_values('tweet_id_x').shape #794,299 back-and-forth dialog responded from companies

(794299, 14)

## citation: https://www.kaggle.com/soaxelbrooke/twitter-basic-seq2seq
## next step 1 - text preprocessing
- Lower casing
- Removal of Punctuations
- Removal of Stopwords
- Removal of Frequent words
- Removal of Rare words
- Stemming
- Lemmatization
- Removal of emojis
- Removal of emoticons
- Conversion of emoticons to words
- Conversion of emojis to words
- Removal of URLs
- Removal of HTML tags
- Chat words conversion
- Spelling correction

## next step 2 - build virtualenv for all necessary packages